# Notebook dùng để crawl thông tin các playlist của những user tương ứng

### Import các thư viện cần dùng

In [1]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv

### Setup trước khi crawl

In [2]:
sleep_time = 1 # Đơn vị: giây

In [3]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

### Định nghĩa hàm crawl các playlist

In [7]:
def get_user_playlist(user_id_file, client_id):
    '''
    Thu thập dữ liệu các playlist của các user tương ứng trong user_id_file
    
    Parameters
    -------
    user_id_file : đường dẫn file .txt chứa danh sách user ID cần 
                   thu thập dữ liệu của các playlist
    client_id : ID truy cập API soundcloud
    
    Returns
    -------
    playlist : DataFrame của các playlist thu được
    '''
    
    list_user_id = np.loadtxt(user_id_file, dtype=int) # User id list
    
    # Khởi tạo playlist dataframe
    playlist = pd.DataFrame(columns = ['id', 'title', 'genre', 'duration', 'likes_count', 'reposts_count',
                                       'is_album', 'release_date', 'created_at', 'last_modified', 'label_name',
                                       'purchase_title', 'purchase_url', 'license', 'tag_list', 'user_id', 'track_count',
                                       'tracks', 'published_at', 'display_date','artwork_url', 'permalink_url'])
    
    # Các Attributes trong playlist dataframe (ngoại trừ thuộc thính 'tracks')
    attributes = ['id', 'title', 'genre', 'duration', 'likes_count', 'reposts_count',
                                       'is_album', 'release_date', 'created_at', 'last_modified', 'label_name',
                                       'purchase_title', 'purchase_url', 'license', 'tag_list', 'user_id', 'track_count',
                                       'published_at', 'display_date','artwork_url', 'permalink_url']

    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[403, 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    # Duyệt từng user id
    for user_id in list_user_id:    
        # Lưu số lượng data đã thu được hiện tại
        # để sau này tránh được bot cài vào api
        checkpoint = len(playlist)
        
        # Lấy request với user_id tương ứng
        url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}&limit=100'
        
        # Ghi log để theo dõi
        print('User id:', user_id)
        print(' Get:', url)
        print('  Status: Doing')
        
        # Kiểm tra user này đã thu thập hết dữ liệu chưa bằng cách 
        # kiểm tra 'next_href' còn dẫn đến ta đến page khác của user không
        this_user_done = False
        while not this_user_done:
            try:
                re = s.get(url)
            except Exception as e:
                # Bỏ qua việc thu thập nếu gặp phải lỗi hoặc chờ quá lâu
                if e != KeyboardInterrupt:
                    print("  Error or timeout! Status code:",re.status_code)
                    print("  Ignore this playlist")
                    continue
                else:
                    return playlist

            # Lặp liên tục đến khi request được
            canGet = False
            while not canGet:
                # Sleep rồi request lại
                if not re.ok:
                    time.sleep(sleep_time)
                    re = requests.get(url)
                    print('  Status: Not ok')
                    continue

                # Check request timeout
                # Nếu requiest timeout, sleep rồi request lại
                if re.status_code == 504:
                    time.sleep(sleep_time)
                    re = requests.get(url)
                    print('  Status: Timeout')
                    continue

                # Vượt ngưỡng số lần cho phép của API -> dừng toàn bộ việc thu thập
                if re.status_code == 429:
                    print('  Status: Exceeds an API rate limit')
                    return playlist

                # Convert Json string thành python dictionary
                json_pydata = json.loads(re.text)

                # Request đã xong
                canGet = True

            # In ra log
            print('  Status: Done')

            # Loop từng playlist của user
            for pl in json_pydata['collection']:
                tracks = ','.join([str(track['id']) for track in pl['tracks']]) # List of tracks in the playlist
                temp = {} # Temporary dictionary để lưu các playlist record hiện tại

                # Lấy thuộc tính tương ứng vào temp
                for key in attributes:
                    temp[key] = pl[key]
                temp['tracks'] = tracks

                # Mở rộng tập dữ liệu vừa thu được
                playlist = playlist.append(temp, ignore_index=True)
               
            # Kiểm tra user này còn page khác không
            if json_pydata['next_href']:
                # Vượt ngưỡng cho phép -> dừng thu thập user này
                if len(playlist) - checkpoint >= 19134:
                    this_user_done = True
                    print(' # current playlists:', len(playlist))
                    continue
                
                # Gán lại url page mới rồi tiếp thục thu thập
                next_url = json_pydata['next_href']
                url = next_url[:next_url.find('offset')] + f'client_id={client_id}&' + next_url[next_url.find('offset'):]
                print(' Get:', url)
                print('  Status: Doing')
            else:
                this_user_done = True
                print(' # current playlists:', len(playlist))
        
    return playlist

### Tiến hành crawl dữ liệu

In [8]:
playlist = get_user_playlist('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

User id: 630253005
 Get: https://api-v2.soundcloud.com/users/630253005/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 0
User id: 217441590
 Get: https://api-v2.soundcloud.com/users/217441590/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 6
User id: 324531068
 Get: https://api-v2.soundcloud.com/users/324531068/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 15
User id: 525378972
 Get: https://api-v2.soundcloud.com/users/525378972/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 22
User id: 9855085
 Get: https://api-v2.soundcloud.com/users/9855085/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 26
User id: 732907996
 Get: https://api-v2.soundcloud.com/u

User id: 204616905
 Get: https://api-v2.soundcloud.com/users/204616905/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 500
User id: 1649030
 Get: https://api-v2.soundcloud.com/users/1649030/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 514
User id: 15275171
 Get: https://api-v2.soundcloud.com/users/15275171/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 514
User id: 84985082
 Get: https://api-v2.soundcloud.com/users/84985082/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 517
User id: 184103480
 Get: https://api-v2.soundcloud.com/users/184103480/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 523
User id: 345125243
 Get: https://api-v2.soundcloud.co

  Status: Done
 # current playlists: 1239
User id: 28418842
 Get: https://api-v2.soundcloud.com/users/28418842/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1239
User id: 387546416
 Get: https://api-v2.soundcloud.com/users/387546416/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1239
User id: 833208370
 Get: https://api-v2.soundcloud.com/users/833208370/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1240
User id: 79723180
 Get: https://api-v2.soundcloud.com/users/79723180/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1240
User id: 944915419
 Get: https://api-v2.soundcloud.com/users/944915419/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1240
Us

User id: 19252023
 Get: https://api-v2.soundcloud.com/users/19252023/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1887
User id: 68515
 Get: https://api-v2.soundcloud.com/users/68515/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1905
User id: 48421686
 Get: https://api-v2.soundcloud.com/users/48421686/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1906
User id: 830729737
 Get: https://api-v2.soundcloud.com/users/830729737/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1907
User id: 18780390
 Get: https://api-v2.soundcloud.com/users/18780390/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1911
User id: 828682924
 Get: https://api-v2.soundcloud.com

  Status: Done
 # current playlists: 1972
User id: 830468146
 Get: https://api-v2.soundcloud.com/users/830468146/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1974
User id: 297125971
 Get: https://api-v2.soundcloud.com/users/297125971/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1977
User id: 1567120
 Get: https://api-v2.soundcloud.com/users/1567120/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1988
User id: 32625393
 Get: https://api-v2.soundcloud.com/users/32625393/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1988
User id: 80588543
 Get: https://api-v2.soundcloud.com/users/80588543/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1988
User i

...

  Status: Done
 # current playlists: 115994
User id: 5723862
 Get: https://api-v2.soundcloud.com/users/5723862/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116003
User id: 88937774
 Get: https://api-v2.soundcloud.com/users/88937774/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116007
User id: 238779498
 Get: https://api-v2.soundcloud.com/users/238779498/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116009
User id: 81724548
 Get: https://api-v2.soundcloud.com/users/81724548/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116036
User id: 682522
 Get: https://api-v2.soundcloud.com/users/682522/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundclo

  Status: Done
 # current playlists: 116757
User id: 255759155
 Get: https://api-v2.soundcloud.com/users/255759155/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116759
User id: 9611971
 Get: https://api-v2.soundcloud.com/users/9611971/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116759
User id: 19552809
 Get: https://api-v2.soundcloud.com/users/19552809/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116763
User id: 818649265
 Get: https://api-v2.soundcloud.com/users/818649265/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 116765
User id: 281135082
 Get: https://api-v2.soundcloud.com/users/281135082/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists:

  Status: Done
 # current playlists: 117263
User id: 827331709
 Get: https://api-v2.soundcloud.com/users/827331709/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117263
User id: 168523531
 Get: https://api-v2.soundcloud.com/users/168523531/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117263
User id: 157675272
 Get: https://api-v2.soundcloud.com/users/157675272/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117274
User id: 465097887
 Get: https://api-v2.soundcloud.com/users/465097887/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117275
User id: 495973866
 Get: https://api-v2.soundcloud.com/users/495973866/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current play

  Status: Done
 # current playlists: 117603
User id: 11150457
 Get: https://api-v2.soundcloud.com/users/11150457/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117613
User id: 132829477
 Get: https://api-v2.soundcloud.com/users/132829477/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117613
User id: 392909973
 Get: https://api-v2.soundcloud.com/users/392909973/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117613
User id: 1334797
 Get: https://api-v2.soundcloud.com/users/1334797/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117613
User id: 54604642
 Get: https://api-v2.soundcloud.com/users/54604642/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 1

  Status: Done
 # current playlists: 117752
User id: 69199909
 Get: https://api-v2.soundcloud.com/users/69199909/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117757
User id: 334147652
 Get: https://api-v2.soundcloud.com/users/334147652/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117796
User id: 48328166
 Get: https://api-v2.soundcloud.com/users/48328166/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117807
User id: 225775374
 Get: https://api-v2.soundcloud.com/users/225775374/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlists: 117808
User id: 139675025
 Get: https://api-v2.soundcloud.com/users/139675025/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 # current playlist

### Lưu kết quả thu được thành file

In [9]:
# Xuất file csv
playlist.to_csv('playlist.csv')

### Đọc file kết quả và xem thử phần đầu tập dữ liệu

In [10]:
# Đọc thử file csv và xem đầu dataframe
df = pd.read_csv('playlist.csv')
df.head()

,Unnamed: 0,id,title,genre,duration,likes_count,reposts_count,is_album,release_date,created_at,...,purchase_url,license,tag_list,user_id,track_count,tracks,published_at,display_date,artwork_url,permalink_url
0,0,1215929743,N3X - D R E A M S,Pop / Tropical House,1100486,11,0,True,2021-03-05T00:00:00Z,2021-02-23T13:45:56Z,...,https://www.toneden.io/freetousemusic/post/dre...,all-rights-reserved,"N3X Dreams ""No copyright music"" ""Royalty free ...",217441590,6,"991225852,991226437,991225747,991228738,991227...",2021-03-05T14:40:25Z,2021-03-05T14:40:25Z,https://i1.sndcdn.com/artworks-XdtJW26VMNnWpip...,https://soundcloud.com/freetousemusic/sets/n3x...
1,1,1211160604,Royalty Free Music,Royalty Free Music,9529286,6,0,False,NaN,2021-02-14T09:35:26Z,...,NaN,all-rights-reserved,"""Royalty Free Music"" ""Free Music For YouTube""",217441590,52,"892455691,957116236,976211350,946099066,897495...",2021-02-14T09:35:26Z,2021-02-14T09:35:26Z,https://i1.sndcdn.com/artworks-UQAgZyIuyPHvAW1...,https://soundcloud.com/freetousemusic/sets/roy...
2,2,1211139496,Copyright Free Music,Copyright Free Music,9529286,5,0,False,NaN,2021-02-14T08:19:07Z,...,NaN,all-rights-reserved,"""Copyright Free Music"" ""Copyright free music f...",217441590,52,"893212930,892455691,954870745,941205094,976211...",2021-02-14T08:19:07Z,2021-02-14T08:19:07Z,https://i1.sndcdn.com/artworks-3K3tVcCZ1nt1LZh...,https://soundcloud.com/freetousemusic/sets/cop...
3,3,1211133652,Background Music For Videos,Background Music For Videos,9529286,6,1,False,NaN,2021-02-14T07:57:14Z,...,NaN,all-rights-reserved,"""Background Music For Videos"" ""Background Musi...",217441590,52,"941194555,933862858,933856318,967357153,892455...",2021-02-14T07:57:14Z,2021-02-14T07:57:14Z,https://i1.sndcdn.com/artworks-p3RvJTMyyQcTSGt...,https://soundcloud.com/freetousemusic/sets/bac...
4,4,1211125735,No Copyright Music,No Copyright Music,9529286,6,0,False,NaN,2021-02-14T07:29:15Z,...,NaN,all-rights-reserved,"""No Copyright Music"" ""Non copyrighted music"" ""...",217441590,52,"976211350,970438177,967357153,957116236,967224...",2021-02-14T07:29:15Z,2021-02-14T07:29:15Z,https://i1.sndcdn.com/artworks-3IkqaxUEh0k4wY4...,https://soundcloud.com/freetousemusic/sets/no-...
